In [ ]:
!pip install openai==0.28
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

In [ ]:
import openai
import os

api_key = 'enter your openai api key'

openai.api_key = api_key

def classify_sentiment(user_input):
    prompt = f"Classify the sentiment of the following text as positive, negative, or neutral: {user_input}"

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You analyze and classify sentiments from text. Return only POSITIVE or NEUTRAL or NEGATIVE"},
            {"role": "user", "content": prompt}
        ]
    )
    sentiment = response['choices'][0]['message']['content'].strip()
    return sentiment

In [ ]:
import pandas as pd
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/EBAC Sem4/practice project/labelled_dataset.csv')
df = df.head(3000)

In [ ]:
rows = []
for _, row in df.iterrows():
    # Tokenize the sentences in the 'body' column
    sentences = sent_tokenize(row['body'])

    # Create a new row for each sentence
    for sentence in sentences:
        rows.append({
            'author': row['author'],
            'created_utc': row['created_utc'],
            'id': row['id'],
            'submission': row['submission'],
            'subreddit': row['subreddit'],
            'subreddit_id': row['subreddit_id'],
            'sentence': sentence
        })

new_df = pd.DataFrame(rows)
len(new_df)

In [ ]:
if 'sentence' in new_df.columns:
    # Initialize tqdm for progress tracking
    tqdm.pandas(desc="Classifying sentiments")

    # Iterate through the DataFrame and classify sentiment for each comment
    new_df['label'] = new_df['sentence'].progress_apply(classify_sentiment)

    print(new_df)
else:
    print("The 'body' column is not found in the dataset.")

In [ ]:
new_df.to_csv('/content/drive/My Drive/Colab Notebooks/EBAC Sem4/practice project/labelled_dataset_sentence_level_clean.csv', index = False)